# Jolpica Lap Dataset Builder

Builds a driver/lap level dataset from the CSV dump stored under `models/jolpica_dump/`. The resulting CSV (`models/driver_lap_dataset.csv`) has one row per driver per lap starting from a configurable `MIN_YEAR`.

## Available data

| Table | Key Columns | Notes |
| --- | --- | --- |
| `formula_one_session.csv` | session metadata (type, timestamp) | No per-lap weather/tyre info |
| `formula_one_round.csv` + `formula_one_season.csv` | season & circuit references | Provide year & circuit IDs |
| `formula_one_sessionentry.csv` | driver entries with grid/classification | Base for dynamic features |
| `formula_one_roundentry.csv` + `formula_one_teamdriver.csv` | link entries to driver/team IDs |  |
| `formula_one_lap.csv` | lap times & positions | No SC/weather data |
| `formula_one_pitstop.csv` | pit stops referencing lap IDs | Used to estimate laps since pit |

Tyre compound, DRS, weather, and safety-car indicators are absent in the dump; the columns are included but left empty for future enrichment.

In [37]:
from pathlib import Path
import pandas as pd
import numpy as np
import bisect

DATA_DIR = Path('jolpica_dump')
OUTPUT_PATH = Path('driver_lap_dataset.csv')
MIN_YEAR = 2012


def slugify(value: str) -> str:
    if not isinstance(value, str):
        return ''
    cleaned = ''.join(ch.lower() if ch.isalnum() else '_' for ch in value)
    return '_'.join(part for part in cleaned.split('_') if part)

In [38]:
def load_table(filename: str) -> pd.DataFrame:
    return pd.read_csv(DATA_DIR / filename)

session_df = load_table('formula_one_session.csv')
round_df = load_table('formula_one_round.csv').rename(columns={'id': 'round_id', 'name': 'round_name'})
season_df = load_table('formula_one_season.csv').rename(columns={'id': 'season_id'})
circuit_df = load_table('formula_one_circuit.csv').rename(columns={'id': 'circuit_id'})
team_df = load_table('formula_one_team.csv').rename(columns={'id': 'team_id'})
lap_df_raw = load_table('formula_one_lap.csv')
session_entry_df = load_table('formula_one_sessionentry.csv').rename(columns={'id': 'session_entry_id'})
round_entry_df = load_table('formula_one_roundentry.csv').rename(columns={'id': 'round_entry_id'})
team_driver_df = load_table('formula_one_teamdriver.csv').rename(columns={'id': 'team_driver_id'})
pitstop_df = load_table('formula_one_pitstop.csv')

In [39]:
session_meta = session_df.merge(
    round_df[['round_id', 'circuit_id', 'season_id', 'round_name']],
    on='round_id', how='left'
)
session_meta = session_meta.merge(
    season_df[['season_id', 'year']],
    on='season_id', how='left'
)
session_meta = session_meta.merge(
    circuit_df[['circuit_id', 'reference']],
    on='circuit_id', how='left'
)
session_meta = session_meta.rename(columns={'reference': 'circuit_reference'})
session_meta['timestamp'] = pd.to_datetime(session_meta['timestamp'], errors='coerce')
session_meta['year_from_timestamp'] = session_meta['timestamp'].dt.year
session_meta['derived_year'] = session_meta['year'].fillna(session_meta['year_from_timestamp'])
session_meta = session_meta[session_meta['derived_year'] >= MIN_YEAR]
session_meta['year'] = session_meta['derived_year']
session_meta['session_name'] = session_meta['round_name'].fillna('Unknown') + ' ' + session_meta['type'].fillna('Session')
session_meta['race_name'] = session_meta['round_name'].fillna('')
session_meta['session_key'] = session_meta.apply(
    lambda r: slugify(f"{int(r['year']) if pd.notna(r['year']) else ''}_{r['round_name']}_{r['type']}"),
    axis=1,
)
session_meta['session_id'] = session_meta['id']
session_meta = session_meta[
    session_meta['type'].str.upper().str.startswith('R')
    | session_meta['session_name'].str.contains('Sprint', case=False, na=False)
]

In [40]:
session_entry = session_entry_df.merge(
    round_entry_df[['round_entry_id', 'team_driver_id']],
    on='round_entry_id', how='left'
)
session_entry = session_entry.merge(
    team_driver_df[['team_driver_id', 'driver_id', 'team_id']],
    on='team_driver_id', how='left'
)
session_entry = session_entry.merge(
    team_df[['team_id', 'name', 'reference']],
    on='team_id', how='left'
)
session_entry = session_entry.rename(columns={'name': 'team_name', 'reference': 'team_reference'})

In [41]:
lap_df = lap_df_raw.rename(columns={'number': 'lap_number', 'position': 'lap_position'})
lap_df = lap_df.merge(
    session_entry[
        ['session_entry_id', 'session_id', 'driver_id', 'team_id', 'team_name', 'team_reference', 'grid']
    ].rename(columns={'grid': 'grid_position'}),
    on='session_entry_id', how='left'
)
lap_df = lap_df.merge(
    session_meta[['session_id', 'year', 'session_name', 'circuit_reference']],
    on='session_id', how='left'
)
lap_df = lap_df.dropna(subset=['year', 'session_name'])

In [42]:
lap_df['lap_number'] = pd.to_numeric(lap_df['lap_number'], errors='coerce')
lap_df['grid_position'] = pd.to_numeric(lap_df['grid_position'], errors='coerce')
lap_df = lap_df.dropna(subset=['lap_number', 'session_id', 'driver_id'])
lap_df['lap_number'] = lap_df['lap_number'].astype(int)

lap_df['lap_time_s'] = pd.to_timedelta(lap_df['time'], errors='coerce').dt.total_seconds()
lap_df.sort_values(['session_entry_id', 'lap_number'], inplace=True)
lap_df['cum_time_s'] = lap_df.groupby('session_entry_id')['lap_time_s'].cumsum()

lap_df['lap_position'] = pd.to_numeric(lap_df['lap_position'], errors='coerce')
lap_df['rank_position'] = lap_df.groupby(['session_id', 'lap_number'])['cum_time_s'].rank(method='first')
lap_df['current_position'] = lap_df['lap_position'].fillna(lap_df['rank_position'])

lap_df['leader_cum_time'] = lap_df.groupby(['session_id', 'lap_number'])['cum_time_s'].transform('min')
lap_df['gap_to_leader_s'] = lap_df['cum_time_s'] - lap_df['leader_cum_time']

lap_df = lap_df.sort_values(['session_id', 'lap_number', 'cum_time_s'])
lap_df['gap_to_ahead_s'] = lap_df.groupby(['session_id', 'lap_number'])['cum_time_s'].diff().fillna(0)

In [43]:
lap_counts = lap_df.groupby('session_id')['lap_number'].max().rename('total_laps')
session_meta = session_meta.merge(lap_counts, on='session_id', how='left')
for col in ['year', 'session_name', 'circuit_reference', 'session_key', 'race_name']:
    lap_df = lap_df.drop(columns=[col], errors='ignore')
lap_df = lap_df.merge(
    session_meta[
        ['session_id', 'year', 'session_name', 'circuit_reference', 'total_laps', 'session_key', 'race_name']
    ],
    on='session_id',
    how='left'
)
# Ensure weather columns exist even if empty
for col in ['humidity', 'pressure', 'rainfall', 'wind_speed', 'wind_direction']:
    if col not in lap_df.columns:
        lap_df[col] = pd.NA

In [ ]:
lap_lookup = lap_df_raw[['id', 'session_entry_id', 'number']].rename(columns={'id': 'lap_id', 'number': 'lap_number'})
lap_lookup['lap_number'] = pd.to_numeric(lap_lookup['lap_number'], errors='coerce')
lap_lookup = lap_lookup.dropna(subset=['lap_number'])
lap_lookup['lap_number'] = lap_lookup['lap_number'].astype(int)
pit_with_lap = pitstop_df.merge(
    lap_lookup[['lap_id', 'session_entry_id', 'lap_number']],
    on='lap_id', how='left'
)
pit_with_lap = pit_with_lap.rename(columns={'session_entry_id_x': 'session_entry_id'})
pit_with_lap = pit_with_lap.dropna(subset=['session_entry_id', 'lap_number'])
pit_with_lap['lap_number'] = pit_with_lap['lap_number'].astype(int)
pit_map = {entry: sorted(group['lap_number'].tolist()) for entry, group in pit_with_lap.groupby('session_entry_id')}
pit_set = {(int(r.session_entry_id), int(r.lap_number)) for r in pit_with_lap.itertuples(index=False)}

def laps_since_pit(entry_id, lap_number):
    laps = pit_map.get(entry_id)
    if not laps:
        return lap_number
    idx = bisect.bisect_right(laps, lap_number)
    last = laps[idx - 1] if idx else 0
    return lap_number - last

lap_df['laps_on_current_tyre'] = [
    laps_since_pit(entry_id, lap_num)
    for entry_id, lap_num in zip(lap_df['session_entry_id'], lap_df['lap_number'])
]

lap_df['is_pit'] = [
    (entry_id, lap_num) in pit_set
    for entry_id, lap_num in zip(lap_df['session_entry_id'], lap_df['lap_number'])
]

lap_df['tyre_compound'] = pd.NA
lap_df['safety_car_this_lap'] = pd.NA
lap_df['virtual_sc_this_lap'] = False
lap_df['drs_enabled'] = pd.NA
lap_df['track_temperature'] = pd.NA
lap_df['air_temperature'] = pd.NA
lap_df['has_rain'] = pd.NA
lap_df['circuit_id'] = lap_df.get('circuit_reference', pd.Series('', index=lap_df.index)).fillna('')

In [ ]:
final_df = pd.DataFrame({
    'driver_id': lap_df['driver_id'],
    'team_id': lap_df['team_id'],
    'team_name': lap_df['team_name'],
    'circuit_id': lap_df['circuit_id'],
    'total_race_laps': lap_df['total_laps'],
    'year': lap_df['year'],
    'session_key': lap_df['session_key'],
    'race_name': lap_df['race_name'],
    'session_name': lap_df['session_name'],
    'grid_position': lap_df['grid_position'],
    'current_position': lap_df['current_position'],
    'gap_to_leader_s': lap_df['gap_to_leader_s'],
    'gap_to_ahead_s': lap_df['gap_to_ahead_s'],
    'lap_time_s': lap_df['lap_time_s'],
    'laps_on_current_tyre': lap_df['laps_on_current_tyre'],
    'tyre_compound': lap_df['tyre_compound'],
    'safety_car_this_lap': lap_df['safety_car_this_lap'],
    'virtual_sc_this_lap': lap_df['virtual_sc_this_lap'],
    'lap_number': lap_df['lap_number'],
    'drs_enabled': lap_df['drs_enabled'],
    'track_temperature': lap_df['track_temperature'],
    'air_temperature': lap_df['air_temperature'],
    'has_rain': lap_df['has_rain'],
    'humidity': lap_df['humidity'],
    'pressure': lap_df['pressure'],
    'rainfall': lap_df['rainfall'],
    'wind_speed': lap_df['wind_speed'],
    'wind_direction': lap_df['wind_direction'],
    'is_pit': lap_df['is_pit'],
})
final_df.sort_values(['year', 'session_key', 'driver_id', 'lap_number'], inplace=True)
final_df.to_csv(OUTPUT_PATH, index=False)
final_df.head()